In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Create the connection engine
engine = create_engine("sqlite:///Resources/strains.sqlite")
conn = engine.connect()

In [3]:
medical_effects = pd.read_sql('SELECT Strains."name" as strain_name, \
	 MedicalEffects."medical_effect" as "medical_effects", ROUND(AVG(MedicalEffects.rating),2) as medical_rating \
FROM Strains \
INNER JOIN MedicalEffects \
ON Strains.id=MedicalEffects.strain_id \
GROUP BY name',conn)

In [4]:
medical_effects

,strain_name,medical_effects,medical_rating
0,00 Kush,Autoimmune Diseases and Inflammation,5.00
1,1024,Pain,4.00
2,2046,Epilepsy,4.50
3,4 Best 4 You,Psychiatric Symptoms,3.00
4,5G's Red x Sour Diesel,Nausea and Vomiting,5.00
...,...,...,...
266,Wild Thailand,Nausea and Vomiting,2.00
267,X-Dog,Pain,4.00
268,X-Kush #2,Nausea and Vomiting,5.00
269,Y Griega,Pain,2.63


In [5]:
strains = pd.read_sql("SELECT id AS strain_id, name as strain_name, breeder as breeder_id FROM Strains",conn)

In [6]:
strains.head()

,strain_id,strain_name,breeder_id
0,1,Af-Pak,1
1,2,00 Cheese,2
2,3,Alien BubbleGum,3
3,4,Cherry OG Kush,4
4,5,Ak 420,5


In [7]:
breeders = pd.read_sql("SELECT id as breeder_id, name as breeder_name FROM Breeders", conn)

In [8]:
breeders.head()

,breeder_id,breeder_name
0,1,210Beans
1,2,00 Seeds Bank
2,3,207 Seeds
3,4,420 Seeds
4,5,420 Genetics


In [9]:
# join strains with breeders on breeder id
strains_breeders = pd.merge(strains,breeders,how='inner',on='breeder_id')

In [10]:
strains_breeders.head()

,strain_id,strain_name,breeder_id,breeder_name
0,1,Af-Pak,1,210Beans
1,8,BlackberryKush,1,210Beans
2,13,Pak-Man,1,210Beans
3,2,00 Cheese,2,00 Seeds Bank
4,12,00 Kush,2,00 Seeds Bank


In [11]:
#Number of strains produced by more than 1 breeder
num_dups = len(strains_breeders.strain_name) - strains_breeders.strain_name.nunique()
num_dups

1672

In [12]:
#medical_effects = pd.read_sql("SELECT strain_id, name as medical_effect, info as medical_info, rating as medical_rating FROM MedicalEffects", conn)
#medical_effects
medical_effects = pd.read_sql("SELECT id as name_id, name as medical_effect, info as medical_info, rating as medical_rating FROM MedicalEffects", conn)

OperationalError: (sqlite3.OperationalError) no such column: name
[SQL: SELECT id as name_id, name as medical_effect, info as medical_info, rating as medical_rating FROM MedicalEffects]
(Background on this error at: http://sqlalche.me/e/13/e3q8)